In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector

In [3]:
mydb = mysql.connector.connect(host="10.50.202.242",
                               user="user01",
                               password="theteam246",
                               database="user01"
                              )

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM sensorrecords_trial")
myresult = mycursor.fetchall()

In [4]:
columns=["index", "device1", "distance1", "device2", "distance2", "date", "time"]
data = pd.DataFrame(myresult, columns=columns).drop(["index"], axis=1)

In [12]:
data.tail(50)

device1  distance1  device2  distance2        date            time  \
50        1      10.85        2      11.12  2020-11-17 0 days 11:09:38   
51        1      10.79        2      11.12  2020-11-17 0 days 11:09:38   
52        1      10.85        2      11.05  2020-11-17 0 days 11:09:38   
53        1      10.79        2      11.05  2020-11-17 0 days 11:09:38   
54        1      10.79        2      11.18  2020-11-17 0 days 11:09:38   
55        1      10.79        2      10.99  2020-11-17 0 days 11:09:38   
56        1      10.79        2      11.12  2020-11-17 0 days 11:09:38   
57        1      10.79        2      11.05  2020-11-17 0 days 11:09:39   
58        1      10.85        2      11.05  2020-11-17 0 days 11:09:39   
59        1      10.85        2      11.05  2020-11-17 0 days 11:09:39   
60        1      10.85        2      11.05  2020-11-17 0 days 11:09:39   
61        1      10.85        2      11.12  2020-11-17 0 days 11:09:39   
62        1      10.79        2      11.18  2020-11-17 0 days 11:09:39   
63        1      10.79        2      11.12  2020-11-17 0 days 11:09:39   
64        1      10.85        2      11.12  2020-11-17 0 days 11:09:40   
65        1      10.79        2      11.05  2020-11-17 0 days 11:09:40   
66        1      10.85        2      11.12  2020-11-17 0 days 11:09:40   
67        1      10.92        2      11.05  2020-11-17 0 days 11:09:40   
68        1      10.92        2      11.05  2020-11-17 0 days 11:09:40   
69        1      10.85        2      10.79  2020-11-17 0 days 11:09:40   
70        1      10.85        2       5.68  2020-11-17 0 days 11:09:40   
71        1       3.30        2      11.18  2020-11-17 0 days 11:09:40   
72        1      10.79        2      11.05  2020-11-17 0 days 11:09:40   
73        1      10.85        2      11.05  2020-11-17 0 days 11:09:40   
74        1       6.61        2      11.05  2020-11-17 0 days 11:09:40   
75        1       3.89        2      11.12  2020-11-17 0 days 11:09:41   
76        1      10.85        2       4.09  2020-11-17 0 days 11:09:42   
77        1      10.79        2      11.12  2020-11-17 0 days 11:09:43   
78        1      10.85        2      11.05  2020-11-17 0 days 11:09:44   
79        1      10.85        2      11.05  2020-11-17 0 days 11:09:44   
80        1      10.79        2      11.05  2020-11-17 0 days 11:09:44   
81        1      10.85        2       4.89  2020-11-17 0 days 11:09:46   
82        1      10.85        2      11.12  2020-11-17 0 days 11:09:47   
83        1       6.01        2      11.12  2020-11-17 0 days 11:09:47   
84        1      10.79        2      11.05  2020-11-17 0 days 11:09:48   
85        1      10.85        2      11.12  2020-11-17 0 days 11:09:49   
86        1      10.79        2      11.05  2020-11-17 0 days 11:09:50   
87        1      10.85        2      11.05  2020-11-17 0 days 11:09:50   
88        1       4.62        2      11.18  2020-11-17 0 days 11:09:51   
89        1      10.39        2      10.72  2020-11-17 0 days 11:09:51   
90        1      10.79        2       9.79  2020-11-17 0 days 11:09:52   
91        1      10.85        2      11.12  2020-11-17 0 days 11:09:53   
92        1      10.79        2      10.92  2020-11-17 0 days 11:09:53   
93        1      10.85        2      11.18  2020-11-17 0 days 11:09:55   
94        1      10.79        2      11.12  2020-11-17 0 days 11:09:55   
95        1      10.85        2       7.21  2020-11-17 0 days 11:09:56   
96        1       3.30        2      11.12  2020-11-17 0 days 11:09:57   
97        1      10.79        2      11.12  2020-11-17 0 days 11:09:58   
98        1      10.85        2      11.05  2020-11-17 0 days 11:09:58   
99        1      10.85        2      11.05  2020-11-17 0 days 11:09:59   

   detected_dist1 detected_dist2  counter  
50          False          False        0  
51          False          False        0  
52          False          False        0  
53          False          False        0  
54       

In [6]:
def counter(data, threshold=5, period=4):
    """
    data: dataframe imported from databse
    threshold: change in distance to be considered
    period: time period we should evaluate for each movement. 6 means calculation checks previous 6 rows
    """
    data["detected_dist1"]=""
    data["detected_dist2"]=""
    data["counter"]=0
    
    for i in range(len(data)):
        previous=max(0, i-period)
        if data["distance1"].iloc[i]<threshold: #motion detected
            data["detected_dist1"].iloc[i]=True
            if data["detected_dist2"].iloc[previous:i].any(): #motion previously detected by device2
                data["counter"].iloc[i]=data["counter"].iloc[i-1]-1 #someone is leaving
            else:
                data["counter"].iloc[i]=data["counter"].iloc[i-1]
        else:
            data["detected_dist1"].iloc[i]=False
            data["counter"].iloc[i]=data["counter"].iloc[i-1]

        if data["distance2"].iloc[i]<threshold: #motion detected
            data["detected_dist2"].iloc[i]=True
            if data["detected_dist1"].iloc[previous:i].any(): #motion previously detected by device2
                data["counter"].iloc[i]=data["counter"].iloc[i-1]+1 #someone is entering
            else:
                data["counter"].iloc[i]=data["counter"].iloc[i-1]
        else:
            data["detected_dist2"].iloc[i]=False
    
    return data

In [15]:
counter(data, threshold=7, period=4)

C:\Users\eda_c\Anaconda3_n\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


device1  distance1  device2  distance2        date            time  \
0         1      10.85        2      11.05  2020-11-17 0 days 11:09:32   
1         1      10.85        2      10.99  2020-11-17 0 days 11:09:32   
2         1      10.85        2      11.05  2020-11-17 0 days 11:09:32   
3         1      10.85        2      11.05  2020-11-17 0 days 11:09:32   
4         1      10.79        2      11.05  2020-11-17 0 days 11:09:33   
..      ...        ...      ...        ...         ...             ...   
95        1      10.85        2       7.21  2020-11-17 0 days 11:09:56   
96        1       3.30        2      11.12  2020-11-17 0 days 11:09:57   
97        1      10.79        2      11.12  2020-11-17 0 days 11:09:58   
98        1      10.85        2      11.05  2020-11-17 0 days 11:09:58   
99        1      10.85        2      11.05  2020-11-17 0 days 11:09:59   

   detected_dist1 detected_dist2  counter  
0           False          False        0  
1           False          False        0  
2           False          False        0  
3           False          False        0  
4           False          False        0  
..            ...            ...      ...  
95          False          False       -2  
96           True          False       -2  
97          False          False       -2  
98          False          False       -2  
99          False          False       -2  

[100 rows x 9 columns]

In [16]:
fig=plt.plot(data.time, data.counter)
fig.savefig("plot.png")